##########       Analyzing Amazon product review data set  using dataframe    #######################

In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/cloudera/parcels/Anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .appName("AmazonReviewDemo") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
# Reading data from S3 to a DataFrame
reviews_df = spark.read.json('s3a://sparkbucket123/Electronics_5.json')

In [ ]:
reviews_df.show(5)

In [ ]:
reviews_df.printSchema()

In [ ]:
# Finding the number of individual reviewers
distinct_reviewers = reviews_df.select("reviewerID").distinct()
distinct_reviewers.count()

In [ ]:
# Finding the number of product counts
reviews_df.select("asin").distinct().count()


In [ ]:
# Find the products with reviews
 reviews_df.select("asin").groupBy("asin").count().orderBy("count" , ascending =False)


In [ ]:
# Find the number of reviews given by each user
reviews_df.select("reviewerID").groupBy("reviewerID").count().orderBy("count" , ascending =False).show()

In [ ]:
# Number of user reviews every month sorted in decreasing order of reviews count

from pyspark.sql.functions import *
reviews_df1= reviews_df.withColumn( "ts" , reviews_df['unixReviewTime'].cast('timestamp'))
reviews_df1.printSchema()


In [ ]:
reviews_df_with_year= reviews_df1.withColumn("year" , year("ts")).withColumn("month" , month("ts"))

reviews_df_with_year.groupBy("year", "month").count().orderBy('count', ascending = False).show()

In [ ]:
# Find how many times each prodcut has been reviewed great

In [ ]:
great_reviews = reviews_df.filter(reviews_df.reviewText.contains("great"))
great_reviews.groupBy("asin").count().orderBy('count', ascending = False).show()
